---
title: "Fine-tuned BERT Model for Sentiment Classification
"
format: html
freeze: auto
jupyter: python3
---





## Objective

In this section,  we fine-tune a **BERT model** for **entity-level sentiment classification** on tweets.

This model serves as our primary benchmark for comparison with the logistic regression baseline and for subsequent interpretability analysis using **LIME**.

---

## Load Dataset and Preprocess
In this section, we load the training and validation datasets, remove empty tweets, and map sentiment labels to integers: Positive → 0, Neutral → 1, Negative → 2, Irrelevant → 3. This prepares the data for model training.


In [ ]:
import pandas as pd
import pandas as pd

train_df = pd.read_csv("data/twitter_training.csv", header=None, names=["id", "entity", "sentiment", "tweet"])
val_df = pd.read_csv("data/twitter_validation.csv", header=None, names=["id", "entity", "sentiment", "tweet"])

# Ensure no missing tweets
train_df = train_df.dropna(subset=["tweet"])
val_df = val_df.dropna(subset=["tweet"])

train_df = train_df[train_df["tweet"].str.strip().astype(bool)]
val_df = val_df[val_df["tweet"].str.strip().astype(bool)]

# Map sentiment to label
label_map = {
    "Positive": 0,
    "Neutral": 1,
    "Negative": 2,
    "Irrelevant": 3
}

train_df["label"] = train_df["sentiment"].map(label_map)
val_df["label"] = val_df["sentiment"].map(label_map)

## Tokenization with HuggingFace
We use Hugging Face’s AutoTokenizer to tokenize tweets into model-ready inputs, including input_ids and attention_mask. The dataset is split and preprocessed in batches using the datasets library.


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["tweet"], padding="max_length", truncation=True, max_length=128)

from datasets import Dataset
dataset = Dataset.from_pandas(train_df[["tweet", "label"]])
dataset = dataset.train_test_split(test_size=0.2)
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.map(lambda x: {"label": int(x["label"])})
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


## Model Setup: BERT + Classification Head
We load a pretrained bert-base-uncased model with a classification head (AutoModelForSequenceClassification) to perform four-class sentiment prediction. The model is moved to GPU or CPU depending on availability.


In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

## Training with Trainer API
The model is fine-tuned using Hugging Face’s Trainer API, which simplifies training and evaluation by managing data loading, loss computation, gradient updates, and metric reporting.


In [ ]:
from transformers import TrainingArguments, Trainer, EvalPrediction 
from  sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report


def compute_metrics(pred):
    preds = pred.predictions.argmax(axis=-1)
    labels = pred.label_ids
    acc = accuracy_score(labels, preds)
    prf = precision_recall_fscore_support(labels, preds, average='macro')
    return {
        "accuracy": acc,
        "precision": prf[0],
        "recall": prf[1],
        "f1": prf[2]
    }


training_args = TrainingArguments(
    output_dir="./bert_model",
    eval_strategy="epoch", 
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)


#trainer.train()


## Save Model


In [ ]:
#trainer.evaluate()

# Save model and tokenizer
#trainer.save_pretrained("scripts/bert_model4")           
#tokenizer.save_pretrained("scripts/bert_model4")

## Evaluate & Save Model


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    "scripts/bert_model4", local_files_only=True
)
tokenizer = AutoTokenizer.from_pretrained("scripts/bert_model4", local_files_only=True)

model.to("cuda")

trainer = Trainer(
    model=model,
    args=training_args,           # TrainingArguments
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

trainer.evaluate()

Here is a comparison of the BERT model vs. the Logistic Regression (TF-IDF) baseline mode


In [ ]:
import matplotlib.pyplot as plt

# Metrics and values
metrics = ["Accuracy", "Precision", "Recall", "F1-score"]
logreg_scores = [0.74, 0.74, 0.74, 0.74]
bert_scores = [0.968, 0.969, 0.967, 0.968]

x = range(len(metrics))
bar_width = 0.35

# Plot
plt.figure(figsize=(8, 5))
plt.bar([i - bar_width/2 for i in x], logreg_scores, width=bar_width, label="Logistic Regression (TF-IDF)")
plt.bar([i + bar_width/2 for i in x], bert_scores, width=bar_width, label="BERT (Fine-Tuned)")

plt.xticks(x, metrics)
plt.ylabel("Score")
plt.ylim(0.6, 1.0)
plt.title("Performance Comparison: Logistic Regression vs. BERT")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Key Findings:
  Significant Accuracy Gain: BERT achieves a ~23% improvement in accuracy   compared to the baseline, indicating better overall predictions.

  Balanced Precision and Recall: BERT maintains both high precision and    recall, suggesting it not only makes correct predictions but also        captures more relevant instances.

  Superior F1-score: The improvement in F1-score demonstrates better       generalization and balance between false positives and false negatives.
